### Calculo de errores para los 3 casos
## Errores 
1. Error de manera individual:
  * Puedo calcular $d_{2D}$ usando **geopy.distance.geodesic**.
  * Luego calcular la diferencia de alturas $\Delta z$
  * Finalmente usar un $d_{3D} = \sqrt{(d_{2D})^2 + (\Delta z)^2}$ y así obtendría mi distancia en 3D.
2. ECM
  * Para obtener el Error Cuadrático Medio puedo, sumar los valores y dividir por la cantidad de puntos (100 puntos)
3. RECM(RMSE)
  * Por último se podría tomar la raiz del ECM para expresar el error promedio en metros.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow.keras.models import load_model
import pickle
from Utils import utils_nn as utlnn
from Utils import geo_conversions as gc

In [2]:
# Carga conjunto de Test.
x_test = pd.read_excel("./Train_Test/Dataset_Separado/x_test_new.xlsx")
y_test = pd.read_excel("./Train_Test/Dataset_Separado/y_test_new.xlsx")

# Quita de columnas no usadas en Train 
x_test = x_test.drop(columns=['latitude_pos_tx', 'longitude_pos_tx', 'elevation_pos_tx', 'year'])

In [32]:
# Cargamos los modelos.
case = 1	# Puede ser 1, 2 o 3, dependiendo del modelo que se quiera cargar.
  
if case == 1: 
	model_x =  load_model('./modelos_entrenamiento/mod_x_R0/modelo1_x_con_scaler/modelo1_x.keras')
	with open('./modelos_entrenamiento/mod_x_R0/modelo1_x_con_scaler/scaler_x1.pkl', 'rb') as f:
		scaler_x = pickle.load(f)

	model_y = load_model('./modelos_entrenamiento/mod_y_R0/modelo1_y_con_scaler/modelo1_y.keras')
	with open('./modelos_entrenamiento/mod_y_R0/modelo1_y_con_scaler/scaler_y1.pkl', 'rb') as f:
		scaler_y = pickle.load(f)
		
	model_z = load_model('./modelos_entrenamiento/mod_z_R0/modelo1/modelo1.keras')
	with open('./modelos_entrenamiento/mod_z_R0/modelo1/scaler_z1.pkl','rb') as f:
		scaler_z = pickle.load(f)

if case == 2: 
    model_z = load_model('./modelos_entrenamiento/mod_z_R0/modelo2/modelo2.keras')
    with open('./modelos_entrenamiento/mod_z_R0/modelo2/scaler_z2.pkl','rb') as f:
        scaler_z = pickle.load(f)

    model_xy = load_model('./modelos_entrenamiento/mod_x_y_R0/modelo1_x_y_con_scaler/mod_x_y_R0.keras')
    with open('./modelos_entrenamiento/mod_x_y_R0/modelo1_x_y_con_scaler/scaler_x.pkl', 'rb') as f:
        scaler_x = pickle.load(f)
    with open('./modelos_entrenamiento/mod_x_y_R0/modelo1_x_y_con_scaler/scaler_y.pkl', 'rb') as f:
        scaler_y = pickle.load(f)

if case == 3:
      model_xyz = load_model('./modelos_entrenamiento/mod_x_y_z_R0/modelo2/modelo2_xyz.keras')
      with open('./modelos_entrenamiento/mod_x_y_z_R0/modelo1/scaler_x.pkl', 'rb') as f:
          scaler_x = pickle.load(f)
      with open('./modelos_entrenamiento/mod_x_y_z_R0/modelo1/scaler_y.pkl', 'rb') as f:
          scaler_y = pickle.load(f)
      with open('./modelos_entrenamiento/mod_x_y_z_R0/modelo1/scaler_z.pkl', 'rb') as f:
          scaler_z = pickle.load(f)

In [33]:
# Predicción para el conjunto Test completo, descalado de MinMax_scaler y escalado a R0
R0	= 6.371E6
if case == 1:
	y_pred_scaled_x = model_x.predict(x_test)
	y_pred_scaled_y = model_y.predict(x_test)
	y_pred_scaled_z = model_z.predict(x_test)
if case == 2: 
	y_pred_scaled_x, y_pred_scaled_y = model_xy.predict(x_test)
	y_pred_scaled_z = model_z.predict(x_test)
if case == 3:
	y_pred_scaled_x, y_pred_scaled_y, y_pred_scaled_z = model_xyz.predict(x_test)

# Desescalado de MinMaxScaler
y_pred_x = scaler_x.inverse_transform(y_pred_scaled_x) * R0
y_pred_y = scaler_y.inverse_transform(y_pred_scaled_y) * R0
y_pred_z = scaler_z.inverse_transform(y_pred_scaled_z) * R0

# Obtención de muestras verdaderasdaderas
y_true_x = y_test.iloc[:,  0:100].to_numpy()
y_true_y = y_test.iloc[:,100:200].to_numpy()
y_true_z = y_test.iloc[:,200:300].to_numpy()

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [34]:
x_test.iloc[[10, 40, 100, 500, 700, 880]] # Son las muestras que se guardan.

,fc [Mhz],elevation,azimuth,mmdd,hour
10,21,34,91,1215,12
40,10,5,98,1010,20
100,10,5,98,928,0
500,25,24,88,1215,12
700,5,0,89,1215,12
880,10,5,98,501,0


In [35]:
lat_true, lon_true, alt_true = gc.transform_spherical_to_geographic(*gc.transform_cartesian_to_spherical(y_true_x, y_true_y, y_true_z))
lat_pred, lon_pred, alt_pred = gc.transform_spherical_to_geographic(*gc.transform_cartesian_to_spherical(y_pred_x, y_pred_y, y_pred_z)) # Altura en metros

In [ ]:
# distances_2d = np.array(list(map(lambda args: np.array(utlnn.distances_by_geodesic(*args)), zip(lat_true, lon_true, lat_pred, lon_pred))))
# distances_3d = np.array(list(map(lambda args: np.array(utlnn.distances_3D(*args)), zip(distances_2d,(alt_true-R0)/1E3,(alt_pred-R0)/1E3))))
# error_2d = np.array(list(map(lambda args: np.array(utlnn.ecm_recm(args)),distances_2d)))
# error_3d =  np.array(list(map(lambda args: np.array(utlnn.ecm_recm(args)),distances_3d)))
# df_errors_2d = pd.DataFrame(error_2d, columns = ['ECM_2D_case3', 'RECM_2D_case3']).round(3)
# df_errors_3d = pd.DataFrame(error_3d, columns=	['ECM_3D_case3', 'RECM_3D_case3']).round(3)
# df_errors = pd.concat([df_errors_2d, df_errors_3d], axis=1)
# df_eval = pd.read_excel('Errores_Evaluacion_Caso_1_2_3.xlsx')
# df_eval = pd.concat([df_eval,df_errors], axis=1)
# df_eval.to_excel('Errores_Evaluacion_Caso_1_2_3.xlsx', index = False)

# Distancia Fréchet
Métrica que mide la similitud entre dos curvas o trayectorias, teniendo en cuenta la forma y el orden de los puntos.

In [46]:
idx = 880 # Indices [10, 40, 100, 500, 700, 880]

lat_true[idx]
lon_true[idx]
alt_true[idx]
lat_pred[idx]
lon_pred[idx]
alt_pred[idx]

trayectoria_true = np.array([lat_true[idx], lon_true[idx], alt_true[idx]])
trayectoria_pred = np.array([lat_pred[idx], lon_pred[idx], alt_pred[idx]])

In [47]:
import numpy as np
np.savetxt(f'./Trayectorias-text/modelo1_x_y_z_r0/trayectoria_true_idx_{idx}.txt', trayectoria_true, delimiter=',', header='latitud, longitud, altitud', comments='')
np.savetxt(f'./Trayectorias-text/modelo1_x_y_z_r0/trayectoria_pred_idx_{idx}.txt', trayectoria_pred, delimiter=',', header='latitud, longitud, altitud', comments='')

In [ ]:
import numpy as np
# Nota: lat[grados] lon[grados] alt[m] radio tierra considerado 6371E3 [m]
tray_true = np.loadtxt('./Trayectorias-text/trayectoria_true_idx_40.txt', delimiter=',', skiprows=1)
tray_pred= np.loadtxt('./Trayectorias-text/trayectoria_pred_idx_40.txt', delimiter=',', skiprows=1)

In [ ]:
# Hasta aquí solo se guardo una muestra de las N existentes.